In [1]:
from preference_generator import create_all_instances, calculate_total_demand, create_noisy_model_student_list, calculate_true_bundle_value
from preference_generator_utils import load_obj
import numpy as np

## Generating Student Instances

### Generating Some True Student Lists

In [2]:
true_student_lists_all_instances, capacities_all_instances, timetables_all_instances = create_all_instances(number_of_instances = 10, number_of_courses= 25, supply_ratio= 1.25, number_of_popular= 9, 
    save_results= True, save_folder= "instances")

Generating students for run 0
Generating students for run 1
Generating students for run 2
Generating students for run 3
Generating students for run 4
Generating students for run 5
Generating students for run 6
Generating students for run 7
Generating students for run 8
Generating students for run 9


In [3]:
def load_instance(instance_number, supply_ratio = 1.25, number_of_popular = 9, large_grid = True, instance_folder = "instances"):
    """
    A simple function to load a single instance, given its number, the supply ration, and the number of pupular courses
    """

    true_student_list = load_obj(f'{instance_folder}/true_student_lists_sr_{supply_ratio}_popular_{number_of_popular}_lg_{large_grid}')[instance_number]
    capacities = np.load(f'{instance_folder}/capacities_all_runs_sr_{supply_ratio}_popular_{number_of_popular}_lg_{large_grid}.npy')[instance_number]
    timetable = load_obj(f'{instance_folder}/timetables_sr_{supply_ratio}_popular_{number_of_popular}_lg_{large_grid}')[instance_number]

    return true_student_list, capacities, timetable


In [4]:
true_student_list, capacities, timetable = load_instance(instance_number = 2)

## Creating a "Noisy" version of a student list

In [5]:
number_of_popular = 9 

# these are the noise profiles that were used in our paper and matched as closely as possible the noise profile in the Budish and Kessler paper. 
if number_of_popular == 9:
    noise_parameter_dictionary = {'noisy_forget_base': 0.5, 'noisy_forget_adjustments': 0.48, 'noisy_base_std': 23, 'noisy_adj_std': 0.2}

elif number_of_popular == 6:
    noise_parameter_dictionary = {'noisy_forget_base': 0.5, 'noisy_forget_adjustments': 0.4825, 'noisy_base_std': 17, 'noisy_adj_std': 0.2}


noisy_student_list = create_noisy_model_student_list(student_list = true_student_lists_all_instances[2], model_type = 'PairwiseAdjustmentsNoisy', 
                                                    seed = 1213, model_param_dictionary = noise_parameter_dictionary)

Noisy all students got called with forget base uniform: 0


## Getting the demands of individual students for a given price vector

In [6]:
price_vector = np.array([0.2 for _ in range(25)])  # A price vector for the individual courses
credit_units = np.array([1 for _ in range(25)])  # the credit units of inidividual courses.

# Demands with respect to their noisy reports in the mechanism
total_demand, individual_demands = calculate_total_demand(prices = np.array([0.2 for _ in range(25)]), student_profiles = noisy_student_list, course_timetable = [[] for  _ in range(5)],
                            credit_units = np.array([1 for _ in range(25)]), model_type = 'PairwiseAdjustmentsNoisy', courses_per_student = 5, return_individual_demands = True)
    
# Demands with respect to their true reports (note: these preferences are incompatible with the CM GUI language)
total_demand_true, individual_demands_true = calculate_total_demand(prices = np.array([0.2 for _ in range(25)]), student_profiles = true_student_list, course_timetable = [[] for  _ in range(5)],
                        credit_units = np.array([1 for _ in range(25)]), model_type = 'True', courses_per_student = 5, return_individual_demands = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-21


## Querying the true value of a student for a bundle

In [7]:
for i in range(len(individual_demands)): 
    value_bundle_noisy_preferences = calculate_true_bundle_value(bundle = individual_demands[i], student_preferences = true_student_list[i], timetable = [[] for  _ in range(5)], make_monotone = True)
    value_bundle_true_preferences = calculate_true_bundle_value(bundle = individual_demands_true[i], student_preferences = true_student_list[i], timetable = [[] for  _ in range(5)], make_monotone = True)

    print(f'For student {i}, value for the optimal bundle {value_bundle_true_preferences} and the one she got under noisy reports: {value_bundle_noisy_preferences}')

For student 0, value for the optimal bundle 386.2362943472781 and the one she got under noisy reports: 302.23872998960667
For student 1, value for the optimal bundle 396.7543121187704 and the one she got under noisy reports: 356.1575381297953
For student 2, value for the optimal bundle 418.2265760730745 and the one she got under noisy reports: 372.143997224518
For student 3, value for the optimal bundle 358.88561807598495 and the one she got under noisy reports: 265.66926424133464
For student 4, value for the optimal bundle 379.9503980064402 and the one she got under noisy reports: 245.0077004942768
For student 5, value for the optimal bundle 378.37980436746705 and the one she got under noisy reports: 261.29276926383284
For student 6, value for the optimal bundle 412.5615875373878 and the one she got under noisy reports: 295.71582775286504
For student 7, value for the optimal bundle 427.7514910074204 and the one she got under noisy reports: 385.3619728989603
For student 8, value for th